In [1]:
import numpy as np
import networkx as nx
from bnn_package import pos_nodes_uniform, connexion_normal_deterministic, add_passive_nodes, evolve_system,step_fhn_rk4
import matplotlib.pyplot as plt

import matplotlib.animation as animation
from matplotlib.colors import Normalize

import matplotlib

matplotlib.use('Qt5Agg')


In [2]:
# ------------ Parameters ------------

#Seed
rng = np.random.default_rng(42)

# Graph parameters 
N_nodes = 2000
std = 0.25
mean = 0.0
(xmax, ymax) = (10.0, 10.0)
f = 0.7 # mean of the poisson of passive nodes per active node

# Evolution parameters : FN 
A = 3.          #0
alpha = 0.2     #1
Eps = 0.08      #2
K = 0.25        #3
V_RP = 1.5      #4
dt = 0.1        #5
params_FN = [A, alpha, Eps,  K,  V_RP, dt]

# Simulation parameters
T_max = 10000
C_r = 1.5
D = 0.1
eps = 0

# Initial conditions
State_0 = np.zeros((N_nodes, 3))
State_0[:, 0] = 0.1 + 0.1 * np.random.randn(N_nodes)  # v_e
State_0[:, 1] = 0.3 + 0.1 * np.random.randn(N_nodes)  # g
State_0[:, 2] = 1.0 + 0.1 * np.random.randn(N_nodes)




In [3]:
# ------------ Build Graph ------------

# Generate active nodes
rng = np.random.default_rng(42)
pos_nodes = pos_nodes_uniform(N_nodes, xmax, ymax, rng)
Adj = connexion_normal_deterministic(pos_nodes, rng, std)
G = nx.from_numpy_array(Adj)
print(pos_nodes.shape)

# Genererate passive nodes
Graph_passive, N_p = add_passive_nodes(G, f,rng)


# Creating pos dictionary for plotting
pos = {}
for i, node in enumerate(G.nodes()):
    pos[node] = (pos_nodes[0, i], pos_nodes[1, i])
print(pos)


# Plot the graph
plt.figure(figsize=(8, 6))
nx.draw(
    Graph_passive,
    pos,
    with_labels=False,
    node_size=100, 
    font_size=8)
labels = {node: str(Graph_passive.nodes[node]["passives"]) for node in G.nodes()}
nx.draw_networkx_labels(
    Graph_passive,
    pos,
    labels=labels,
    font_color='white',
    font_size=8,
    font_weight='bold'
)
plt.axis('equal')
plt.xlim(0, xmax)  
plt.ylim(0, ymax)
plt.title("Graph with Passive Nodes")
plt.show()



Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


(2, 2000)
{0: (np.float64(7.739560485559633), np.float64(8.42731968806472)), 1: (np.float64(4.388784397520523), np.float64(4.463894921437095)), 2: (np.float64(8.585979199113824), np.float64(9.529751615601034)), 3: (np.float64(6.973680290593639), np.float64(6.507945897177367)), 4: (np.float64(0.9417734788764953), np.float64(1.1589886531924942)), 5: (np.float64(9.756223516367559), np.float64(8.850929882268007)), 6: (np.float64(7.61139701990353), np.float64(4.892651908307978)), 7: (np.float64(7.860643052769538), np.float64(1.4163896797290498)), 8: (np.float64(1.2811363267554587), np.float64(1.5269013384142072)), 9: (np.float64(4.503859378955672), np.float64(6.875339351098718)), 10: (np.float64(3.7079802423258124), np.float64(0.47372506729489783)), 11: (np.float64(9.267649888486018), np.float64(2.8659376491381328)), 12: (np.float64(6.438651200806645), np.float64(1.397875904698681)), 13: (np.float64(8.227616132708299), np.float64(4.410774296172843)), 14: (np.float64(4.4341419882733115), np.

In [4]:
# ------------ Evolution ------------
Adj = Adj.astype(np.float64)
Trajectory = evolve_system(State_0, T_max, params_FN, step_fhn_rk4, eps, N_p, Adj, C_r, D)

AttributeError: module 'tqdm' has no attribute '__call__'

In [133]:
# ------------ Some plot of some nodes ---------------
plt.figure(figsize=(12, 6))
v_e = Trajectory[:, 4:7, 0]
plt.plot( np.arange(len(v_e[:,0])),v_e,label=f'Node {4}')
plt.plot( np.arange(len(v_e[:,1])),v_e,label=f'Node {5}')
plt.plot( np.arange(len(v_e[:,2])),v_e,label=f'Node {6}')

plt.xlabel('Time Step')
plt.ylabel('Voltage $V_e$')
plt.legend()
plt.grid(True)
plt.show()
print(v_e[:7,:])


[[0.164877   0.04779984 0.11810227]
 [0.12310278 0.17636802 0.08653906]
 [0.12823141 0.2468376  0.1003796 ]
 [0.14563429 0.28851799 0.12161787]
 [0.16363338 0.31495844 0.14041806]
 [0.17868731 0.33293158 0.15496427]
 [0.19008432 0.34596855 0.16550812]]


In [160]:
activity = Trajectory[:, :, 0]  # Forme (T, N)

# Positions des nœuds (pos est un dictionnaire {node: (x, y)})
nodes = list(pos.keys())
x_pos = [pos[node][0] for node in nodes] 
y_pos = [pos[node][1] for node in nodes]  

# Normalisation des valeurs pour la colormap
norm = Normalize(vmin=activity.min(), vmax=activity.max())

# ------------ Initialisation de la figure ------------
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(
    x_pos, y_pos,
    c=activity[0, :],  
    cmap='viridis',    
    norm=norm,
    s=100,             
    edgecolor='k',     
    linewidth=0.5
)

# Ajouter une colorbar pour la légende
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Activité du nœud (dimension 0)')

# Titre et labels
ax.set_title('Heatmap de l\'activité des nœuds au fil du temps')
ax.set_xlabel('Position X')
ax.set_ylabel('Position Y')

# ------------ Fonction de mise à jour pour l'animation ------------
def update(frame):
    """Met à jour les couleurs des points pour la frame donnée."""
    scatter.set_array(activity[frame*4, :])  # Met à jour les couleurs
    ax.set_title(f'Heatmap - Temps = {frame}')
    return scatter,

# ------------ Création de l'animation ------------
n_frames = activity.shape[0]//4

# Créer l'animation
anim = animation.FuncAnimation(
    fig,
    update,
    frames=n_frames,
    interval=0.1, 
    blit=False   
)


'''
# ------------ Sauvegarde de la vidéo ------------
# Choisissez un format: 'mp4', 'gif', etc.
anim.save(
    'node_activity_heatmap.mp4',
    writer='ffmpeg',  # Nécessite ffmpeg (install: `conda install ffmpeg` ou `brew install ffmpeg`)
    fps=10,           # Images par seconde
    dpi=300           # Résolution
)

'''
plt.show()
plt.close()  #